<a href="https://colab.research.google.com/github/ARAN1218/EngineerOffers_AI/blob/main/jupyter_notebook/%E3%82%A8%E3%83%B3%E3%82%B8%E3%83%8B%E3%82%A2%E6%B1%82%E4%BA%BAAI(TECH_STOCK)_%E3%83%87%E3%83%BC%E3%82%BF%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pprint import pprint
pd.set_option('display.max_rows', None)

df = pd.read_pickle('Engineer_Agent_Bigdata.pickle')

In [ ]:
df.head(10)

In [ ]:
for column in df.columns:
    display(df[column].value_counts())

## ページ

In [ ]:
df_page = df.copy()
df_page['ページ']

In [ ]:
#ページはいらないので削ぎ落とし一択
del df_page['ページ']
df_page.head(1)

In [ ]:
def page_preprocessing(df_page):
    del df_page['ページ']
    
    print('page preprocessed!')

In [ ]:
#test
df_page = df.copy()
page_preprocessing(df_page)
df_page.head(1)

## タイトル

In [ ]:
df_title = df.copy()
df_title['タイトル']

In [ ]:
#とりあえずタグを除去してみる
df_title['タイトル'] = df_title['タイトル'].str.translate(str.maketrans({'\n':''}))
df_title['タイトル']

In [ ]:
pprint(df_title['タイトル'].value_counts())
# 使わない

In [ ]:
def title_preprocessing(df_title):
    del df_title['タイトル']
    print("title preprocessed!")

## 月額報酬

In [ ]:
df_reward = df.copy()
df_reward['月額報酬'].value_counts()

In [ ]:
#00~00の形だけ残す
df_reward['月額報酬'] = df_reward['月額報酬'].str.translate(str.maketrans({'\n':'', '万':'', '円':'', '(':'', ')':'', '税':'', '別':''}))
df_reward['月額報酬']

In [ ]:
#足して2で割る
df_reward['月額報酬'] = df_reward['月額報酬'].map(lambda x : (int(x.split('～')[0])+int(x.split('～')[1]))/2 if ('～' in x) else int(x))
df_reward['月額報酬']

In [ ]:
def reward_preprocessing(df_reward):
    #00~00の形だけ残す
    df_reward['月額報酬'] = df_reward['月額報酬'].str.translate(str.maketrans({'\n':'', '万':'', '円':'', '(':'', ')':'', '税':'', '別':''}))

    #足して2で割る
    df_reward['月額報酬'] = df_reward['月額報酬'].map(lambda x : (int(x.split('～')[0])+int(x.split('～')[1]))/2 if ('～' in x) else int(x))

    print('reward preprocessed!')

In [ ]:
#test
df_reward = df.copy()
reward_preprocessing(df_reward)
df_reward['月額報酬'].value_counts()

## 勤務地

In [ ]:
df_office = df.copy()
df_office['勤務地'].unique()
#鍵は「/」

In [ ]:
#タグ等を除去する
df_office['勤務地'] = df_office['勤務地'].str.translate(str.maketrans({'\n':''
                                                                 , ' ':''
                                                                 , '\u3000':''
                                                                 ,'・':''
                                                                 #,'':''
                                                                 , '│':'/'
                                                                 , '／':'/'
                                                                 ,'+':'/'
                                                                 ,'＋':'/'
                                                                 ,'※':'/'
                                                                 ,'(':'/'
                                                                 ,'（':'/'
                                                                }))
df_office['勤務地'].unique()

In [ ]:
#「/」で区切って、左の要素を取得する。
df_office['勤務地'] = df_office['勤務地'].map(lambda x : x.split('/')[0] if ('/' in x) else x)
df_office['勤務地'].unique()

In [ ]:
#地名からリモートを消去し、リモートを表す言葉を「リモート」で統一したい
eraces = [
    'フルリモート'
    ,'現状リモート勤務'
    ,'、リモート勤務'
    ,'基本リモート'
    ,'原則リモート'
    ,'リモート勤務'
    ,'一部リモート'
    ,'、リモート'
    ,'テレワーク'
    ,'orリモート'
    ,'リモート、'
    ,'リモート'
]

for erace in eraces:
    df_office['勤務地'] = df_office['勤務地'].str.replace(erace, '')
df_office['勤務地'][df_office['勤務地']==''] = 'リモート'
df_office['勤務地'].unique()

In [ ]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

LE_office = LabelEncoder()
LE_office.fit(df_office['勤務地'])
df_office['勤務地'] = LE_office.transform(df_office['勤務地'])
df_office['勤務地'].unique()

In [ ]:
def office_preprocessing(df_office):
    #タグ等を除去する
    df_office['勤務地'] = df_office['勤務地'].str.translate(str.maketrans({'\n':''
                                                                     , ' ':''
                                                                     , '\u3000':''
                                                                     ,'・':''
                                                                     #,'':''
                                                                     , '│':'/'
                                                                     , '／':'/'
                                                                     ,'+':'/'
                                                                     ,'＋':'/'
                                                                     ,'※':'/'
                                                                     ,'(':'/'
                                                                     ,'（':'/'
                                                                    }))
    
    #「/」で区切って、左の要素を取得する。
    df_office['勤務地'] = df_office['勤務地'].map(lambda x : x.split('/')[0] if ('/' in x) else x)

    #地名からリモートを消去し、リモートを表す言葉を「リモート」で統一したい
    eraces = [
        'フルリモート'
        ,'現状リモート勤務'
        ,'、リモート勤務'
        ,'基本リモート'
        ,'原則リモート'
        ,'リモート勤務'
        ,'一部リモート'
        ,'、リモート'
        ,'テレワーク'
        ,'orリモート'
        ,'リモート、'
        ,'リモート'
    ]

    for erace in eraces:
        df_office['勤務地'] = df_office['勤務地'].str.replace(erace, '')
    df_office['勤務地'][df_office['勤務地']==''] = 'リモート'
    
    #ラベルエンコーディング
    from sklearn.preprocessing import LabelEncoder

    global LE_office
    LE_office = LabelEncoder()
    LE_office.fit(df_office['勤務地'])
    df_office['勤務地'] = LE_office.transform(df_office['勤務地'])
    
    print('office preprocessed!')

In [ ]:
#test
df_office = df.copy()
office_preprocessing(df_office)
print(LE_office.classes_)
df_office['勤務地'].unique()

## 勤務期間

In [ ]:
df_span = df.copy()
df_span['勤務期間'].value_counts()
#延長フラグ作る...?
#後 - 前　で期間の長さ計算しよかな
#長期 = 999
#即日 = 0(8月1日)

In [ ]:
#延長カラムを作っていく
#「延長」「更新」という単語が含まれている行は1、それ以外は0にする
df_span['延長'] = df_span['勤務期間'].map(lambda x : 1 if (('延長' in x) or ('更新' in x)) else 0)
df_span.head(10)

In [ ]:
# 即日を2021年7月長期を2024年7月(=3年)と定義し、変換する
df_span['勤務期間'] = df_span['勤務期間'].str.replace('即日', '2021年7月')
df_span['勤務期間'] = df_span['勤務期間'].str.replace('長期', '2024年7月')

In [ ]:
#不要な文字を削除する。
df_span['勤務期間'] = df_span['勤務期間'].str.translate(str.maketrans({'\n':''
                                                               , ' ':''
                                                               ,'月':'時'
                                                               ,'中':''
                                                               , '(':'|'
                                                               ,'~':'～'
                                                              }))
df_span['勤務期間'].unique()

In [ ]:
#「|」の左側だけ切り取る
df_span['勤務期間'] = df_span['勤務期間'].map(lambda x : x.split('|')[0] if ('|' in x) else x)
df_span['勤務期間'].unique()

In [ ]:
#「～」で分けて、両方の要素を加工して足した値を特徴量とする。

df_span['1'] = df_span['勤務期間'].map(lambda x : x.split('～')[0] if ('～' in x) else x)
df_span['2'] = df_span['勤務期間'].map(lambda x : x.split('～')[1] if ('～' in x) else 0)
df_span['1'].unique()

In [ ]:
#「時」の左側だけ切り取る
df_span['1'] = df_span['1'].map(lambda x : x.split('時')[0] if ('時' in str(x)) else x)
df_span['2'] = df_span['2'].map(lambda x : x.split('時')[0] if ('時' in str(x)) else x)
df_span['1'].unique()

In [ ]:
# 悔しいけど個別に対応していく
# 不明な文字列は2022年9月に変換
# 数値はstrにして2021年と足す
df_span['1'][df_span['1'] == '8'] = '2021年8'
df_span['1'][df_span['1'] == '2021/9/1'] = '2021年9'
df_span['2'][df_span['2'] == '2021/9/30'] = '2021年9'
df_span['2'] = df_span['2'].apply(lambda x : '2021年'+str(x) if x in ['7', '8', '9', '10', '11', '12'] else x)
df_span['2'] = df_span['2'].apply(lambda x : '2022年7' if x in ['', '終了', '1年想定', '確認', 0] else x)

In [ ]:
display(df_span['1'].value_counts())
display(df_span['2'].value_counts())

In [ ]:
# datetime型にして月単位で差をとる
from datetime import datetime
for i in range(len(df_span)):
    df_span['勤務期間'][i] = ((datetime.strptime(df_span['2'][i], '%Y年%m') - datetime.strptime(df_span['1'][i], '%Y年%m')).days + 1) // 30

df_span['勤務期間']

In [ ]:
df_span[['勤務期間', '1', '2']]

In [ ]:
df_span.dtypes

In [ ]:
def span_preprocessing(df_span):
    #延長カラムを作っていく
    #「延長」「更新」という単語が含まれている行は1、それ以外は0にする
    df_span['延長'] = df_span['勤務期間'].map(lambda x : 1 if (('延長' in x) or ('更新' in x)) else 0)
    
    # 即日を2021年7月長期を2024年7月(=3年)と定義し、変換する
    df_span['勤務期間'] = df_span['勤務期間'].str.replace('即日', '2021年7月')
    df_span['勤務期間'] = df_span['勤務期間'].str.replace('長期', '2024年7月')
    
    #不要な文字を削除する。
    df_span['勤務期間'] = df_span['勤務期間'].str.translate(str.maketrans({'\n':''
                                                                   , ' ':''
                                                                   ,'月':'時'
                                                                   ,'中':''
                                                                   , '(':'|'
                                                                   ,'~':'～'
                                                                  }))
    
    #「|」の左側だけ切り取る
    df_span['勤務期間'] = df_span['勤務期間'].map(lambda x : x.split('|')[0] if ('|' in x) else x)
    
    #「～」で分ける。
    df_span['1'] = df_span['勤務期間'].map(lambda x : x.split('～')[0] if ('～' in x) else x)
    df_span['2'] = df_span['勤務期間'].map(lambda x : x.split('～')[1] if ('～' in x) else 0)
    
    #「時」の左側だけ切り取る
    df_span['1'] = df_span['1'].map(lambda x : x.split('時')[0] if ('時' in str(x)) else x)
    df_span['2'] = df_span['2'].map(lambda x : x.split('時')[0] if ('時' in str(x)) else x)
    
    # 不明な文字列は2022年9月に変換
    # 数値はstrにして2021年と足す
    df_span['1'][df_span['1'] == '8'] = '2021年8'
    df_span['1'][df_span['1'] == '2021/9/1'] = '2021年9'
    df_span['2'][df_span['2'] == '2021/9/30'] = '2021年9'
    df_span['2'] = df_span['2'].apply(lambda x : '2021年'+str(x) if x in ['7', '8', '9', '10', '11', '12'] else x)
    df_span['2'] = df_span['2'].apply(lambda x : '2022年7' if x in ['', '終了', '1年想定', '確認', 0] else x)
    
    # datetime型にして月単位で差をとる
    from datetime import datetime
    for i in range(len(df_span)):
        df_span['勤務期間'][i] = ((datetime.strptime(df_span['2'][i], '%Y年%m') - datetime.strptime(df_span['1'][i], '%Y年%m')).days + 1) // 30
        
    del df_span['1']
    del df_span['2']

    print("span preprocessed!")


In [ ]:
# テスト
df_span = df.copy()
span_preprocessing(df_span)
df_span

## 稼働率

In [ ]:
df_per = df.copy()
df_per['稼働率'].value_counts()

In [ ]:
#余計な文字列を取り除く
df_per['稼働率'] = df_per['稼働率'].str.translate(str.maketrans({'\n':'', ' ':'', '%':''}))
df_per['稼働率'].value_counts()

In [ ]:
#「～」で分割する
df_per['1'] = df_per['稼働率'].map(lambda x : int(x.split('～')[0]) if ('～' in x) else int(x))
df_per['2'] = df_per['稼働率'].map(lambda x : int(x.split('～')[1]) if ('～' in x) else int(x))

display(df_per['1'].value_counts())
display(df_per['2'].value_counts())

In [ ]:
#1と2の平均値を特徴量とする
df_per['稼働率'] = (df_per['1'] + df_per['2']) / 2
df_per['稼働率'].value_counts()

In [ ]:
def per_preprocessing(df_per):
    #余計な文字列を取り除く
    df_per['稼働率'] = df_per['稼働率'].str.translate(str.maketrans({'\n':'', ' ':'', '%':''}))

    #「～」で分割する
    df_per['1'] = df_per['稼働率'].map(lambda x : int(x.split('～')[0]) if ('～' in x) else int(x))
    df_per['2'] = df_per['稼働率'].map(lambda x : int(x.split('～')[1]) if ('～' in x) else int(x))

    #1と2の平均値を特徴量とする
    df_per['稼働率'] = (df_per['1'] + df_per['2']) / 2
    
    del df_per['1']
    del df_per['2']
    
    print('per preprocessed!')

In [ ]:
#test
df_per = df.copy()
per_preprocessing(df_per)
df_per['稼働率'].value_counts()

## 業務内容

In [ ]:
df_task = df.copy()
df_task['業務内容']
# もしかしたら使わないかもしれない

In [ ]:
def task_preprocessing(df_task):
    del df_task['業務内容']
    
    print("task preprocessed!")


## スキル

In [ ]:
df_skill = df.copy()
df_skill['スキル']

In [ ]:
# 要らなそうな文字を消してみる
df_skill['スキル'] = df_skill['スキル'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_skill['スキル'] = df_skill['スキル'].str.replace('【必須スキル】', '')
df_skill['スキル']

In [ ]:
df_skill['スキル'].value_counts()

In [ ]:
def skill_preprocessing(df_skill):
    del df_skill['スキル']
    
    print("skill preprocessed!")

## 業種

In [ ]:
df_type = df.copy()
df_type['業種']

In [ ]:
df_type['業種'] = df_type['業種'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_type['業種'].unique()

In [ ]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

LE_type = LabelEncoder()
LE_type.fit(df_type['業種'])
df_type['業種'] = LE_type.transform(df_type['業種'])
df_type['業種'].unique()

In [ ]:
def type_preprocessing(df_type):
    df_type['業種'] = df_type['業種'].str.translate(str.maketrans({'\n':'', ' ':''}))
    
    #ラベルエンコーディング
    from sklearn.preprocessing import LabelEncoder

    LE_type = LabelEncoder()
    LE_type.fit(df_type['業種'])
    df_type['業種'] = LE_type.transform(df_type['業種'])
    
    print('type preprocessed!')

In [ ]:
# テスト
df_type = df.copy()
type_preprocessing(df_type)
df_type['業種']

## リモート

In [ ]:
df_remote = df.copy()
df_remote['リモート']

In [ ]:
df_remote['リモート'] = df_remote['リモート'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_remote['リモート'].unique()

In [ ]:
# ''のデータはリモートなしとして扱う
df_remote['リモート'][df_remote['リモート'] == ''] = 'なし（常駐）'
df_remote['リモート'].unique()

In [ ]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

LE_remote = LabelEncoder()
LE_remote.fit(df_remote['リモート'])
df_remote['リモート'] = LE_remote.transform(df_remote['リモート'])
df_remote['リモート'].unique()

In [ ]:
def remote_preprocessing(df_remote):
    df_remote['リモート'] = df_remote['リモート'].str.translate(str.maketrans({'\n':'', ' ':''}))
    df_remote['リモート'][df_remote['リモート'] == ''] = 'なし（常駐）'
    
    #ラベルエンコーディング
    from sklearn.preprocessing import LabelEncoder

    LE_remote = LabelEncoder()
    LE_remote.fit(df_remote['リモート'])
    df_remote['リモート'] = LE_remote.transform(df_remote['リモート'])
    df_remote['リモート'].unique()
    
    print("remote preprocessed!")


In [ ]:
# テスト
df_remote = df.copy()
remote_preprocessing(df_remote)
df_remote['リモート'].unique()

## 募集職種

In [ ]:
df_job = df.copy()
df_job['募集職種']

In [ ]:
pd.get_dummies(df_job['業種'])

In [ ]:
df_job['募集職種'] = df_job['募集職種'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_job['募集職種'].unique()

In [ ]:
df_job['募集職種'][df_job['募集職種'].isin(['UNIX・C/C++エンジニア'])]

In [ ]:
# ダミー変数化
jobs = [
    'その他オフィスワーク',
    'システムエンジニア・プログラマー',
    'インフラ・ネットワークエンジニア',
    'ネットワークエンジニア',
    'コンサル・PM・PMO',
    'テスター・デバッカー',
    'フロントエンドエンジニア',
    'バックエンドエンジニア',
    'デザイナー・クリエイター',
    'Java系エンジニア',
    '業務系エンジニア',
    'サーバエンジニア',
    '運用/監視担当',
    '企画・マーケティング',
    'データサイエンティスト',
    'DBA(データベース)管理者',
    'ヘルプデスク',
    'スマホアプリ開発',
    'ソーシャル系エンジニア',
    '制御・組み込み系エンジニア',
    'ゲームプログラマー/ゲームクリエイター',
    'アナリスト',
    'カスタマーサポート',
    'Webディレクター',
    'SAP系(ABAP・BASIS)エンジニア',
    'Windows系エンジニア',
    'LAMP系エンジニア'
]

for job in jobs:
    df_job[job] = df_job['募集職種'].map(lambda x : 1 if job in x else 0)
    df_job['募集職種'] = df_job['募集職種'].str.replace(job, '')
    
df_job['募集職種'].value_counts()

In [ ]:
def job_preprocessing(df_job):
    df_job['募集職種'] = df_job['募集職種'].str.translate(str.maketrans({'\n':'', ' ':''}))
    
    # ダミー変数化
    jobs = [
        'その他オフィスワーク',
        'システムエンジニア・プログラマー',
        'インフラ・ネットワークエンジニア',
        'ネットワークエンジニア',
        'コンサル・PM・PMO',
        'テスター・デバッカー',
        'フロントエンドエンジニア',
        'バックエンドエンジニア',
        'デザイナー・クリエイター',
        'Java系エンジニア',
        '業務系エンジニア',
        'サーバエンジニア',
        '運用/監視担当',
        '企画・マーケティング',
        'データサイエンティスト',
        'DBA(データベース)管理者',
        'ヘルプデスク',
        'スマホアプリ開発',
        'ソーシャル系エンジニア',
        '制御・組み込み系エンジニア',
        'ゲームプログラマー/ゲームクリエイター',
        'アナリスト',
        'カスタマーサポート',
        'Webディレクター',
        'SAP系(ABAP・BASIS)エンジニア',
        'Windows系エンジニア',
        'LAMP系エンジニア'
    ]

    for job in jobs:
        df_job[job] = df_job['募集職種'].map(lambda x : 1 if job in x else 0)
        df_job['募集職種'] = df_job['募集職種'].str.replace(job, '')
        
    del df_job['募集職種']
        
    print("job preprocessed!")


In [ ]:
# テスト
df_job = df.copy()
job_preprocessing(df_job)
df_job.head(10)

In [ ]:
df_job.sum()

## ポジション

In [ ]:
df_position = df.copy()
df_position['ポジション']

In [ ]:
df_position['ポジション'] = df_position['ポジション'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_position['ポジション'].unique()

In [ ]:
# ''のデータはメンバーに置き換える
df_position['ポジション'][df_position['ポジション'] == ''] = 'メンバー'

In [ ]:
# メンバーとサブメンバーの数も十分だったので、上書きする方式でフラグカラム作る
# メンバー:1、サブリーダー:1、リーダー:2
df_position['ポジション'] = df_position['ポジション'].str.replace('サブリーダー', 'サブリーダ')
df_position['ポジション'] = df_position['ポジション'].apply(lambda x : 2 if ('リーダー' in x) else 1 if 'サブリーダ' in x else 0)
df_position['ポジション']

In [ ]:
def position_preprocessing(df_position):
    df_position['ポジション'] = df_position['ポジション'].str.translate(str.maketrans({'\n':'', ' ':''}))
    
    # メンバー:1、サブリーダー:1、リーダー:2
    df_position['ポジション'] = df_position['ポジション'].str.replace('サブリーダー', 'サブリーダ')
    df_position['ポジション'] = df_position['ポジション'].apply(lambda x : 2 if ('リーダー' in x) else 1 if 'サブリーダ' in x else 0)
    
    print("position preprocessed!")


In [ ]:
# テスト
df_position = df.copy()
position_preprocessing(df_position)
df_position['ポジション'].unique()

## 募集人数

In [ ]:
df_people = df.copy()
df_people['募集人数']
# NaNって何だよ

In [ ]:
df_people['募集人数'] = df_people['募集人数'].str.translate(str.maketrans({'\n':'', ' ':'', '名':''}))
df_people['募集人数']

In [ ]:
# NaNは1に置き換える
df_people['募集人数'] = df_people['募集人数'].fillna(1)
df_people['募集人数'].value_counts()

In [ ]:
df_people['募集人数'] = df_people['募集人数'].astype(int)
df_people['募集人数']

In [ ]:
def people_preprocessing(df_people):
    df_people['募集人数'] = df_people['募集人数'].str.translate(str.maketrans({'\n':'', ' ':'', '名':''}))
    
    # NaNは1に置き換える
    df_people['募集人数'] = df_people['募集人数'].fillna(1)
    df_people['募集人数'] = df_people['募集人数'].astype(int)
    
    print("people preprocessed!")


In [ ]:
# テスト
df_people = df.copy()
people_preprocessing(df_people)
df_people['募集人数']

## 勤務時間

In [ ]:
df_time = df.copy()
df_time['勤務時間']

In [ ]:
df_time['勤務時間'] = df_time['勤務時間'].str.translate(str.maketrans({'\n':'', ' ':''}))
df_time['勤務時間']

In [ ]:
df_time['勤務時間'].value_counts()
# 全体の5分の1もデータが取れてないので、使わないかも

In [ ]:
def time_preprocessing(df_time):
    del df_time['勤務時間']
    
    print("time preprocessed!")

## 服装

In [ ]:
df_fashion = df.copy()
df_fashion.服装

In [ ]:
df_fashion.服装 = df_fashion.服装.str.translate(str.maketrans({'\n':'', ' ':''}))
df_fashion.服装.value_counts()

In [ ]:
# NaNはビジネスカジュアルとする
df_fashion.服装 = df_fashion.服装.fillna('ビジネスカジュアル')
df_fashion.服装.value_counts()

In [ ]:
# スーツネクタイ着用=0, ビジネスカジュアル=1, 私服可=2
df_fashion.服装[df_fashion['服装'] == 'スーツネクタイ着用'] = 0
df_fashion.服装[df_fashion['服装'] == 'ビジネスカジュアル'] = 1
df_fashion.服装[df_fashion['服装'] == '私服可'] = 2
df_fashion.服装.value_counts()

In [ ]:
def fashion_preprocessing(df_fashion):
    df_fashion.服装 = df_fashion.服装.str.translate(str.maketrans({'\n':'', ' ':''}))
    
    # NaNはビジネスカジュアルとする
    df_fashion.服装 = df_fashion.服装.fillna('ビジネスカジュアル')
    
    # スーツネクタイ着用=0, ビジネスカジュアル=1, 私服可=2
    df_fashion.服装[df_fashion['服装'] == 'スーツネクタイ着用'] = 0
    df_fashion.服装[df_fashion['服装'] == 'ビジネスカジュアル'] = 1
    df_fashion.服装[df_fashion['服装'] == '私服可'] = 2
    
    print("fashion preprocessed!")


In [ ]:
# テスト
df_fashion = df.copy()
fashion_preprocessing(df_fashion)
df_fashion.服装.value_counts()

## 契約形態

In [ ]:
df_contract = df.copy()
df_contract.契約形態

In [ ]:
df_contract.契約形態 = df_contract.契約形態.str.translate(str.maketrans({'\n':'', ' ':''}))
df_contract.契約形態.value_counts()

In [ ]:
# 紹介予定派遣を派遣に組み込む
df_contract.契約形態[df_contract['契約形態'] == '紹介予定派遣'] = '派遣'
df_contract.契約形態.value_counts()

In [ ]:
# 業務委託=0, 派遣=1, 契約社員=2
df_contract.契約形態[df_contract['契約形態'] == '業務委託'] = 0
df_contract.契約形態[df_contract['契約形態'] == '派遣'] = 1
df_contract.契約形態[df_contract['契約形態'] == '契約社員'] = 2
df_contract.契約形態.value_counts()

In [ ]:
def contract_preprocessing(df_contract):
    df_contract.契約形態 = df_contract.契約形態.str.translate(str.maketrans({'\n':'', ' ':''}))
    
    # 紹介予定派遣を派遣に組み込む
    df_contract.契約形態[df_contract['契約形態'] == '紹介予定派遣'] = '派遣'
    
    # 業務委託=0, 派遣=1, 契約社員=2
    df_contract.契約形態[df_contract['契約形態'] == '業務委託'] = 0
    df_contract.契約形態[df_contract['契約形態'] == '派遣'] = 1
    df_contract.契約形態[df_contract['契約形態'] == '契約社員'] = 2
    
    print("contract preprocessed!")


In [ ]:
# テスト
df_contract = df.copy()
contract_preprocessing(df_contract)
df_contract.契約形態.value_counts()

## 備考

In [ ]:
df_sub = df.copy()
df_sub.備考
# 説明変数には使えなそう

In [ ]:
def sub_preprocessing(df_sub):
    del df_sub['備考']
    
    print("sub preprocessed!")

## 開発環境

In [ ]:
df_env = df.copy()
df_env.開発環境

In [ ]:
df_env.開発環境 = df_env.開発環境.str.translate(str.maketrans({'\n':'', ' ':''}))
df_env.開発環境.value_counts()
# 思いの他取得できていないので、多分スキルの方で抽出した方がいい

In [ ]:
def env_preprocessing(df_env):
    del df_env['開発環境']
    
    print("env preprocessed!")

## 英語力

In [ ]:
df_english = df.copy()
df_english.英語力

In [ ]:
df_english.英語力 = df_english.英語力.str.translate(str.maketrans({'\n':'', ' ':''}))
df_english.英語力.value_counts()

In [ ]:
# RLSWのダミー変数を作る

# イテレートするためにNaNを取り除かないとならないらしい
df_english.英語力 = df_english.英語力.fillna('')

for eng in ['Reading', 'Listening', 'Speaking', 'Writing']:
    print(eng)
    df_english[eng] = df_english.英語力.map(lambda x : 1 if eng in x else 0)
    display(df_english[eng].value_counts())
    df_english.英語力 = df_english.英語力.str.replace(eng, '')
    
    
del df_english['英語力']

In [ ]:
def english_preprocessing(df_english):
    df_english.英語力 = df_english.英語力.str.translate(str.maketrans({'\n':'', ' ':''}))
    
    # イテレートするためにNaNを取り除かないとならないらしい
    df_english.英語力 = df_english.英語力.fillna('')

    for eng in ['Reading', 'Listening', 'Speaking', 'Writing']:
        df_english[eng] = df_english.英語力.map(lambda x : 1 if eng in x else 0)
        df_english.英語力 = df_english.英語力.str.replace(eng, '')

    del df_english['英語力']
    
    print("english preprocessed!")


In [ ]:
# テスト
df_english = df.copy()
english_preprocessing(df_english)
df_english